# UNData API Exercise #

### Cavin Jacobson ###

**In this exercise, you'll redo the data gathering phase of the UNData Exploration project by using APIs instead of downloading csv files.**

**You'll make use of the World Bank Indicators API. Note that this API does not require an API key. Before attempting the exercise, it would be a good idea to skim through the Documentation page and to check out the Basic Call Structure article.**

In [4]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json

1. Use the API to get all available data for the GDP per capita, PPP (constant 2017 international $) indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [6]:
params = {
    'per_page': 20000
}

In [7]:
endpointgdp = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD?format=json'

response = requests.get(endpointgdp, params = params)

In [8]:
gdp_res= response.json()

In [77]:
gdp_norm = (
    pd.json_normalize(gdp_res[1])
    .drop(columns = ['unit', 'obs_status', 'country.id', 'indicator.id', 'indicator.value', 'countryiso3code', 'decimal'])
    .rename(columns = {'value': 'GDP Per Capita', 'date': 'Year', 'country.value': 'Country'})
)

In [79]:
gdp_norm.head(3)

,Year,GDP Per Capita,Country
0,2023,4047.007031,Africa Eastern and Southern
1,2022,4038.638689,Africa Eastern and Southern
2,2021,3994.171654,Africa Eastern and Southern


2. Now, use the API to get all available data for Life expectancy at birth, total (years). This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [43]:
endpointle = 'https://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN?format=json'

response_le = requests.get(endpointle, params = params)

In [45]:
le_res= response_le.json()

In [73]:
le_norm = (
    pd.json_normalize(le_res[1])
    .drop(columns = ['unit', 'obs_status', 'country.id', 'indicator.id', 'indicator.value', 'countryiso3code', 'decimal'])
    .rename(columns = {'value': 'Life Expectancy', 'date': 'Year', 'country.value': 'Country'})
)

In [75]:
le_norm.head(3)

,Year,Life Expectancy,Country
0,2023,NaN,Africa Eastern and Southern
1,2022,62.899031,Africa Eastern and Southern
2,2021,62.454590,Africa Eastern and Southern


3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.

In [99]:
pd.merge(gdp_norm[['Year', 'Country', 'GDP Per Capita']], le_norm)

,Year,Country,GDP Per Capita,Life Expectancy
0,2023,Africa Eastern and Southern,4047.007031,NaN
1,2022,Africa Eastern and Southern,4038.638689,62.899031
2,2021,Africa Eastern and Southern,3994.171654,62.454590
3,2020,Africa Eastern and Southern,3919.499230,63.313860
4,2019,Africa Eastern and Southern,4130.057222,63.755678
...,...,...,...,...
17019,1964,Zimbabwe,NaN,54.994000
17020,1963,Zimbabwe,NaN,54.549000
17021,1962,Zimbabwe,NaN,54.071000
17022,1961,Zimbabwe,NaN,53.619000


4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the Country API. Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

## Bonus Questions ##

1. Adjust your request so that it returns data just for the United States 

2. Adjust your request so that it returns data just for the United States for the year 2021.

3. Adjust your request so that it returns data just for the United States for the years 2000 through 2021.

4. Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.

5. If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.

6. The endpoint "http://api.worldbank.org/v2/indicator" will return information about all available indicators. See Indicator API Queries. Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator. 